In [2]:
import pandas as pd
import numpy as np

"""concatenate data"""
dflist = []
for i in range(1,13):
    fn = '../2016_data_new/2016_{}_new.csv'.format(i)
    df = pd.read_csv(fn, index_col=0)
    dflist.append(df)
df = pd.concat(dflist, ignore_index=True)

In [3]:
"""select all flights with ORIGIN at SAN"""
san = df.loc[df['ORIGIN']=='SAN']
print san.head(2)
print san.columns

     YEAR  MONTH  DAY_OF_MONTH  DAY_OF_WEEK     FL_DATE UNIQUE_CARRIER  \
225  2016      1             4            1  2016-01-04             F9   
227  2016      1             4            1  2016-01-04             F9   

     AIRLINE_ID ORIGIN ORIGIN_CITY_NAME ORIGIN_STATE_NM         ...           \
225       20436    SAN    San Diego, CA      California         ...            
227       20436    SAN    San Diego, CA      California         ...            

    DEP_DELAY  ARR_TIME  ARR_DELAY  ACTUAL_ELAPSED_TIME  DISTANCE  \
225      -6.0    1148.0      -17.0                133.0     853.0   
227     -15.0    2235.0      -25.0                130.0     853.0   

     CARRIER_DELAY  WEATHER_DELAY  NAS_DELAY  SECURITY_DELAY  \
225            NaN            NaN        NaN             NaN   
227            NaN            NaN        NaN             NaN   

     LATE_AIRCRAFT_DELAY  
225                  NaN  
227                  NaN  

[2 rows x 22 columns]
Index([u'YEAR', u'MONTH', u'DAY

In [4]:
"""see destination airports"""
grouped = san.groupby(['DEST'])
grp_size = grouped.size().sort_values(ascending=False)
# airports = list(grp_size.index[:25])

airports = ['ATL', 'ORD', 'DEN', 'LAX', 'DFW', 'SFO', 'PHX', 'LAS', 'IAH', 'SEA', 
            'MSP', 'MCO', 'DTW', 'BOS', 'EWR', 'CLT', 'SLC', 'JFK', 'BWI', 'LGA', 
            'MDW', 'FLL', 'SAN', 'DCA', 'PHL']
selected = grp_size.loc[grp_size.index.isin(airports)]
# print grp_size
print selected

DEST
SFO    7760
PHX    5433
DEN    4864
LAS    4440
SEA    3632
DFW    3491
ORD    3143
LAX    3017
SLC    2475
ATL    2263
JFK    2065
IAH    1729
MDW    1299
MSP    1192
EWR    1072
BOS    1032
PHL     908
BWI     794
CLT     787
DTW     786
MCO     680
FLL     195
dtype: int64


In [5]:
selected = san.loc[san['DEST'].isin(airports)]
airlines = selected['UNIQUE_CARRIER'].unique()

In [6]:
selected = san.loc[san['DEST'].isin(airports)]

# print selected.head()
# print selected['DEST'].unique()
# print df.loc[(df['ORIGIN']=='SAN') & (df['DEST']=='DCA')]

"""get count of delays per airport"""
delay_al = {}
for al in airlines:
#     pc_list = []
    delay_m = {}
    for month in range(1,13):
        month_total = selected.loc[(selected['MONTH']==month) & (selected['UNIQUE_CARRIER']==al)]
        delays = month_total.loc[month_total['ARR_DELAY']>0]
        
        num_delay = delays.groupby(['DEST']).size()
        num_total = month_total.groupby(['DEST']).size()
        delay_pc = num_delay.divide(num_total)
        
        ap_used = list(delay_pc.index)
        
        delay_ap = np.zeros((len(airports),))
        for i,ap in enumerate(airports):
            if ap in ap_used:
                delay_ap[i] = delay_pc[ap]
                
        delay_m[month] = delay_ap
        
        
#         delay_m[month] = delay_
        
        
#         pc_list.append(delay_pc)
    #     print 'month:', month
    #     print delay_pc

#     pc_df = pd.concat(pc_list, axis=1)
    delay_al[al] = delay_m

# for iten,val in delay_al.iteritems():
#     print iten
#     print val
#     print



In [9]:
"""average delay time"""
selected = san.loc[san['DEST'].isin(airports)]

delay_al = {}
for al in airlines:
    delay_m = {}
    for month in range(1,13):
        month_total = selected.loc[(selected['MONTH']==month) & (selected['UNIQUE_CARRIER']==al)]
        delays = month_total.loc[month_total['ARR_DELAY']>0]
        
        delay_avg = delays.groupby(['DEST'])['ARR_DELAY'].mean()
#         print delay_avg
#         break
        
        ap_used = list(delay_avg.index)
        delay_ap = np.zeros((len(airports),))
        for i,ap in enumerate(airports):
            if ap in ap_used:
                delay_ap[i] = delay_avg[ap]
                
        delay_m[month] = delay_ap
        
    delay_al[al] = delay_m

# for iten,val in delay_al.iteritems():
#     print iten
#     print val
#     print